In [1]:
import importlib
import dbops_updated as dbops
import cv2
import time
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch

from facelib import FaceDetector, FaceRecognizer, EmotionDetector, get_config
import glob
import math

In [2]:
importlib.reload(dbops)  # Reload the module from source

<module 'dbops_updated' from '/home/u1541638/Projects/Data-Mining-Spring-2025-Project/dbops_updated.py'>

In [3]:
!ls /scratch/u1319464/Data-Mining-Spring-2025-Project/data | grep -v "Dragon Ball"

Bishops Wife 1947.mp4
Day the Earth Stood Still 1951.mp4
Dracula 1931.mp4
Earth vs the Flying Saucers color.mp4
Fight.Club.10th.Anniversary.Edition.1999.1080p.BrRip.x264.YIFY.mp4
Frankenstein 1931 colorized.mp4
Hellhole.1985.1080p.BluRay.H264.AAC-RARBG.mp4
Invasion of the Body Snatchers.mp4
Invisible Man 1933.mp4
It Came from Outer Space 1953 colorized.mp4
Kronos 1957 colorized.mp4
Natural Born Killer A.mp4
Natural Born Killer B.mp4
Sahara 1943.mp4
Tarantula 1955 colorized.mp4
Taxi Driver A.mp4
Taxi Driver B.mp4
The Matrix (1999) [OPEN MATTE] [WEB-DL 1080p 10bit DD5.1 x265].mp4
Wolf Man 1941.mp4


In [4]:
set(glob.glob('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/*mp4')) - set(glob.glob('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Dragon Ball*mp4'))

{'/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Bishops Wife 1947.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Day the Earth Stood Still 1951.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Dracula 1931.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Earth vs the Flying Saucers color.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Fight.Club.10th.Anniversary.Edition.1999.1080p.BrRip.x264.YIFY.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Frankenstein 1931 colorized.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Hellhole.1985.1080p.BluRay.H264.AAC-RARBG.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Invasion of the Body Snatchers.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Invisible Man 1933.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/It Came from Outer Space 1953 colorized.mp4',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project

In [5]:
movie_mapping = {
    "Wolf Man 1941": "The Wolf Man",
    "Kronos 1957 colorized": "Kronos",
    "Hellhole.1985.1080p.BluRay.H264.AAC-RARBG": "Hellhole",
    "Taxi Driver B": "Taxi Driver",
    "Dracula 1931": "Dracula",
    "Natural Born Killer B": "Natural Born Killers",
    "Invisible Man 1933": "The Invisible Man (1933)",
    "The Matrix (1999) [OPEN MATTE] [WEB-DL 1080p 10bit DD5.1 x265]": "The Matrix",
    "Fight.Club.10th.Anniversary.Edition.1999.1080p.BrRip.x264.YIFY": "Fight Club",
    "Sahara 1943": "Sahara",
    "Taxi Driver A": "Taxi Driver",
    "Frankenstein 1931 colorized": "Frankenstein",
    "Day the Earth Stood Still 1951": "The Day the Earth Stood Still",
    "Natural Born Killer A": "Natural Born Killers",
    "Invasion of the Body Snatchers": "Invasion of the Body Snatchers",
    "Bishops Wife 1947": "The Bishop's Wife",
    "It Came from Outer Space 1953 colorized": "It Came from Outer Space",
    "Tarantula 1955 colorized": "Tarantula",
    "Earth vs the Flying Saucers color": "Earth vs. the Flying Saucers"
}


In [6]:
import glob
for file_path in (set(glob.glob('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/*mp4')) - set(glob.glob('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Dragon Ball*mp4'))):
    title = file_path.replace('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/', '').replace('.mp4', '')
    omdb_data = dbops.omdb.get_movie_data(movie_mapping.get(title, title))
    if omdb_data!=None:
        dbops.Movie.add_movie_from_omdb_data(file_path, omdb_data)

In [7]:

# Earth vs the Flying Saucers color
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Earth vs the Flying Saucers color.mp4', dbops.omdb.get_movie_data(identifier='tt0049169', is_imdb_id=True))

# Wolf Man 1941
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Wolf Man 1941.mp4', dbops.omdb.get_movie_data(identifier='tt0034398', is_imdb_id=True))

# Dracula 1931
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Dracula 1931.mp4', dbops.omdb.get_movie_data(identifier='tt0021814', is_imdb_id=True))

# Taxi Driver A
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Taxi Driver A.mp4', dbops.omdb.get_movie_data(identifier='tt0075314', is_imdb_id=True))
# Taxi Driver B
# dbops.Movie.add_movie_from_omdb_data('./data/Earth vs the Flying Saucers color.mp4', dbops.omdb.get_movie_data(identifier='tt0075314', is_imdb_id=True)

# Natural Born Killer A
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Natural Born Killer A.mp4', dbops.omdb.get_movie_data(identifier='tt0110632', is_imdb_id=True))
# Natural Born Killer B
# dbops.omdb.get_movie_data(identifier='tt0110632', is_imdb_id=True)

# Bishops Wife 1947
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Bishops Wife 1947.mp4', dbops.omdb.get_movie_data(identifier='tt0039190', is_imdb_id=True))

# Kronos 1957 colorized
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Kronos 1957 colorized.mp4', dbops.omdb.get_movie_data(identifier='tt0050610', is_imdb_id=True))

# Sahara 1943
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Sahara 1943.mp4', dbops.omdb.get_movie_data(identifier='tt0036323', is_imdb_id=True))

# It Came from Outer Space 1953 colorized
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/It Came from Outer Space 1953 colorized.mp4', dbops.omdb.get_movie_data(identifier='tt0045920', is_imdb_id=True))

# Tarantula 1955 colorized
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Tarantula 1955 colorized.mp4', dbops.omdb.get_movie_data(identifier='tt0048696', is_imdb_id=True))

# The Matrix (1999)
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/The Matrix (1999) [OPEN MATTE] [WEB-DL 1080p 10bit DD5.1 x265].mp4', dbops.omdb.get_movie_data(identifier='tt0133093', is_imdb_id=True))

# Frankenstein 1931 colorized
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Frankenstein 1931 colorized.mp4', dbops.omdb.get_movie_data(identifier='tt0021884', is_imdb_id=True))

# Hellhole.1985.1080p.BluRay.H264.AAC-RARBG
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Hellhole.1985.1080p.BluRay.H264.AAC-RARBG.mp4', dbops.omdb.get_movie_data(identifier='tt0089269', is_imdb_id=True))

# Fight.Club.10th.Anniversary.Edition.1999.1080p.BrRip.x264.YIFY
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Fight.Club.10th.Anniversary.Edition.1999.1080p.BrRip.x264.YIFY.mp4', dbops.omdb.get_movie_data(identifier='tt0137523', is_imdb_id=True))

# Day the Earth Stood Still 1951
dbops.Movie.add_movie_from_omdb_data('/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Day the Earth Stood Still 1951.mp4', dbops.omdb.get_movie_data(identifier='tt0043456', is_imdb_id=True))


In [8]:
movie = dbops.Movie('tt0089269')
movie.get_title(), movie.get_file_path(), movie.get_date_created()

('Hellhole',
 '/scratch/u1319464/Data-Mining-Spring-2025-Project/data/Hellhole.1985.1080p.BluRay.H264.AAC-RARBG.mp4',
 '26 Apr 1985')

In [9]:
face_detector = FaceDetector(name='resnet')
emotion_detector = EmotionDetector()
conf = get_config()
face_regonizer = FaceRecognizer(conf)

from EmotionDetector: weights loaded


In [10]:

face_index_sample_resolution = 15 # seconds
process_frame_subsample_rate = 30 # 1 = process all frames, 2 = process every other frame, 3 = process every thrid frame
# to improve performance, we first build an index of every 15 seconds that tells us whether that frame had a face.
# next, we only process interfavals of 15 seconds which started and/or ended with a frame which had a face
# this avoids processing areas with no faces

num_faces_index = {} # frame_index --> number of faces

def process_frame(frame_ref: dbops.Frame, frame_array=None):

    if frame_array is None:
        frame_array = np.array(frame_ref.get_frame_image())

    frame_ref.compute_frame_aggregated()
    
    # Detect faces
    with torch.no_grad():
        aligned_faces, boxes, scores, landmarks = face_detector.detect_align(frame_array)

        num_faces = len(aligned_faces)
        frame_ref.set_num_faces(num_faces)
        
        num_faces_index[frame_ref.frame_index] = num_faces
    
        # Recognize facial expression (happy, sad, angry, etc.)
        # emotions, probab = emotion_detector.detect_emotion(faces)
        if num_faces>0:
            embs = face_regonizer.feature_extractor(aligned_faces)
            aligned_faces = aligned_faces.permute(0, 3, 1, 2)
            aligned_faces = aligned_faces.float().div(255).cuda()
            
            # should this be softmaxxed?

            emotion_vectors = torch.softmax(emotion_detector.model(aligned_faces), dim=1).cpu().numpy()
            
            for i, (aligned_face, emotion_vector, landmark, emb) in enumerate(zip(aligned_faces.permute(0, 2, 3, 1).cpu().numpy(), emotion_vectors, landmarks.cpu().numpy(), embs)):
                face = dbops.Face.add_face(imdb_id, frame_idx, i, landmark, emotion_vector, emb)
                face.set_face_image_aligned(Image.fromarray(cv2.cvtColor(aligned_face*255, cv2.COLOR_BGR2RGB).clip(0, 255).astype(np.uint8)))
                face.compute_face_aggregated()

        # else:
        #     print(f'No faces found in frame {frame_idx}')

for movie_ref in dbops.Movie.iterate_all_movies():
    imdb_id = movie_ref.get_imdb_id()
    title = movie_ref.get_title()
    file_path = movie_ref.get_file_path()
    
    print(f'Now Processing: {title}')

    video_stream = cv2.VideoCapture(file_path)

    if not video_stream.isOpened():
        print(f"Error: Could not open video {file_path}")
    
    fps = video_stream.get(cv2.CAP_PROP_FPS)       # frames per second
    
    process_frame_subsample_rate = int(fps)

    offset_minutes = 10
    offset_in_frames = int(offset_minutes * 60 * fps)
    video_stream.set(cv2.CAP_PROP_POS_FRAMES, offset_in_frames)
    
    # --- PROCESS next 5 minutes of frames ---
    total_frames = int(video_stream.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_process = total_frames - int(5 * 60 * fps)  # 5 minutes = 300 seconds
    
    # Ensure we don't exceed the total number of frames
    upper_frame_limit = offset_in_frames + frames_to_process
    if upper_frame_limit > total_frames:
        upper_frame_limit = total_frames
    
    print(f"Starting from minute {offset_minutes}, processing up to minute {(upper_frame_limit / 60 / fps):02f}")

    print(f'Building num_faces index of frames from every {face_index_sample_resolution} seconds')

    num_faces_index = {} # clear previous frames
    
    for frame_idx in tqdm(range(offset_in_frames, upper_frame_limit, int(math.ceil(face_index_sample_resolution * fps))), desc='Building Index'):
        frame_ref = dbops.Frame.add_frame(imdb_id, frame_idx)
        video_stream.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = video_stream.read()

        if not ret:
            print(f"Error: Could not read frame: {frame_idx}")
            continue

        # looks hackey but this avoids worring about caching the whole movie somewhere.
        # instead, the movie is streamed right here, and then we get rid of it after
        frame_ref.frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        process_frame(frame_ref, frame)
        
        del frame_ref, frame

    processed_frames = movie_ref.iterate_frames()
    processed_frame_idxs = set( f.frame_index for f in movie_ref.iterate_frames() )

    print('Processing facial scenes')

    video_stream.set(cv2.CAP_PROP_POS_FRAMES, offset_in_frames) # restart video
    
    indexed_frames = list(sorted(num_faces_index.keys()))
    for i in tqdm(range(len(indexed_frames)-1), desc='Processing Frames'):
        
        start_frame = indexed_frames[i]
        end_frame =   indexed_frames[i+1]
        # print(f'Start frame: {start_frame}\tEnd Frame: {end_frame}')
        
        if num_faces_index[start_frame] > 0 or num_faces_index[end_frame] > 0:

            # only compute frames in the current scene, given the downsample rate, and remove any frames which were already processed
            
            total_frames_in_scene = end_frame - start_frame
            frames_to_process = set(range(start_frame, end_frame, process_frame_subsample_rate))
            
            # total_frames_in_subsampled_scene = len(frames_to_process)
            frames_to_process = frames_to_process - processed_frame_idxs

            total_unprocessed_frames_in_subsampled_scene = len(frames_to_process)

            # print(f'Computation reduction from downsampling & caching: {((total_frames_in_scene - total_unprocessed_frames_in_subsampled_scene) / total_frames_in_scene):02f}')
            
            for frame_idx in sorted(frames_to_process):
        
                frame_ref = dbops.Frame.add_frame(imdb_id, frame_idx)
        
                # if frame_ref.is_fully_cached(): # the above check is not full-proof like this but is way faster, and makes this redundant
                #     print(f'Skipped frame {frame_idx}')
                #     continue

                video_stream.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = video_stream.read()
        
                if not ret:
                    print(f"Error: Could not read frame: {frame_idx}")
                    continue
        
                # looks hackey but this avoids worring about caching the whole movie somewhere.
                # instead, the movie is streamed right here, and then we get rid of it after
                frame_ref.frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                
                process_frame(frame_ref, frame)
                
                del frame_ref, frame

        # else:
        #     print('No faces detected by index; Skipping scene.')

    video_stream.release()
    print(f"Processing complete for: {title}")
print("Processing complete!")



Now Processing: Dracula
Starting from minute 10, processing up to minute 74.463312
Building num_faces index of frames from every 15 seconds


Building Index:  24%|██▍       | 63/258 [00:15<00:47,  4.08it/s]


KeyboardInterrupt: 

In [ ]:
start_frame, end_frame

(136425, 136785)

In [ ]:
!du -sh movies.db omdb images data tmp

585M	movies.db
88K	omdb
1.8G	images
5.0M	data
du: cannot access 'tmp': No such file or directory
